In [ ]:
import utils_cubic_Lorenz
import utils_emulator
import numpy as np

%load_ext autoreload
%autoreload 2

## Experiment 4: Cubic Lorenz System (weak nonlinearities)

This notebook runs the coupled three box model for the following scenarios and emulation techniques:

Scenarios:
1. Abrupt - An abrupt doubling of CO2 concentration; corresponds roughly to the *Abrupt2xCO2* CMIP experiment.
2. High Emissions - An exponential increase of CO2 concentration in time; corresponds roughly to *SSP585*.
3. Plateau - An increase in CO2 concentration in time that follows a hyperbolic tangent, increasing exponentially and then tapering off; corresponds roughly to *SSP245*.
4. Overshoot - An increase in CO2 concentration in time that follows a Gaussian profile, increasing and decreasingly rapidly; inspired by *SSP119*, but decreases more quickly.

Emulators:
1. Pattern Scaling - Time-invariant pattern based on linear regression from global mean temperature to local temperature.
2. Fluctuation Dissipation Theorem - Response functions derived through perturbation experiment.
3. Deconvolution - Response functions solved for from any general experiment.
4. Modal Fitting - Response functions fit from any general experiment.
5. Dynamic Mode Decomposition (DMD) - Approximating system dynamics with a linear operator.
6. Extended DMD - Approximating system dynamics with nonlinear basis functions.

#### Setup and Run Scenarios

Required before creating/evaluating emulators.

In [ ]:
# Parameters
sigma, beta  = 10.0, 8 / 3
rho_base     = 28.0
dt           = 0.01
delta_rho    = 1.0/dt
alpha        = 1/1000
t_max_pert   = 5
t_max_scen   = 250
t_vec_pert   = np.arange(0.0, t_max_pert + dt, dt)
t_vec_scen   = np.arange(0.0, t_max_scen + dt, dt)
n_steps_pert = t_vec_pert.size
n_steps_scen = t_vec_scen.size
n_ensemble   = 5_00
eps          = 4
scenarios    = ['Abrupt','High Emissions','Plateau','Overshoot']

rho                         = utils_cubic_Lorenz.gen_rho(t_vec_scen, scenarios)
state0, baseline_mean       = utils_cubic_Lorenz.spin_up(n_ensemble, rho_base, sigma, alpha, beta, dt, eps)
state_ensemble, state_mean  = utils_cubic_Lorenz.run_Lorenz(scenarios, state0, n_ensemble, n_steps_scen, rho, sigma, alpha, beta, dt, eps)
z_ensemble, z_mean, z_std   = utils_cubic_Lorenz.get_z_vals(scenarios, open=False, state_ensemble=state_ensemble, state_mean=state_mean)

### Method I: Pattern Scaling

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
NRMSE_all_PS = utils_emulator.evaluate_ensemble('PS', z_ensemble, z_mean, None, rho, scenarios, n_ensemble, n_choices, cubLor=True)

if save_error:
  utils_emulator.save_results(NRMSE_all_PS, 'exp4_I_PS_error_ensemble')

### Method II: FDT

In [ ]:
R_FDT, state_base_ensemble, state_pert_ensemble = utils_cubic_Lorenz.run_Lorenz_pert(state0, n_ensemble, n_steps_pert, rho_base, sigma, alpha, beta, dt, eps)
z_base_ensemble, z_pert_ensemble = state_base_ensemble[:,:,2], state_pert_ensemble[:,:,2]

save_error = False

n_choices = 10 # Number of subsamples to take
delta = 50 * dt
NRMSE_all_FDT = utils_emulator.evaluate_ensemble('FDT', z_ensemble, z_mean, None, rho, scenarios, n_ensemble,
                                                 n_choices, dt=dt, cubLor=True, rho_base=rho_base, t_vec=t_vec_scen,
                                                 baseline_mean=baseline_mean, z_base_ensemble=z_base_ensemble,
                                                 z_pert_ensemble=z_pert_ensemble, delta=delta)

if save_error:
  utils_emulator.save_results(NRMSE_all_FDT, 'exp4_II_FDT_error_ensemble')

### Method III: Deconvolution

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
NRMSE_all_deconvolve = utils_emulator.evaluate_ensemble('deconvolve', z_ensemble, z_mean, None, rho, scenarios, n_ensemble,
                                                        n_choices, dt=dt, cubLor=True, rho_base=rho_base, t_vec=t_vec_scen,
                                                        baseline_mean=baseline_mean)

if save_error:
  utils_emulator.save_results(NRMSE_all_deconvolve, 'exp4_III_deconv_error_ensemble')

### Method IV: Modal Fit

In [ ]:
save_error = False

n_choices = 10 # Number of subsamples to take
n_modes, n_boxes = 1, 1
NRMSE_all_modal = utils_emulator.evaluate_ensemble('modal', z_ensemble, z_mean, None, rho, scenarios, n_ensemble,
                                                   n_choices, dt=dt, cubLor=True, rho_base=rho_base, t_vec=t_vec_scen,
                                                   baseline_mean=baseline_mean, n_modes=n_modes, n_boxes=n_boxes)

if save_error:
  utils_emulator.save_results(NRMSE_all_modal, 'exp4_IV_modal_error_ensemble')

### Method V: DMD

In [ ]:
save_error = False

n_choices = 10                # Number of subsamples to take
n_steps   = len(t_vec_scen)            # No. timesteps
w0        = np.array([baseline_mean[2]]) # Initial condition
NRMSE_all_DMD = utils_emulator.evaluate_ensemble('DMD', z_ensemble, z_mean, None, rho, scenarios, n_ensemble,
                                                 n_choices, dt=dt, cubLor=True, rho_base=rho_base, t_vec=t_vec_scen,
                                                 baseline_mean=baseline_mean, n_boxes=n_boxes, w0=w0, n_steps=n_steps)

if save_error:
  utils_emulator.save_results(NRMSE_all_DMD, 'exp4_V_DMD_error_ensemble')

### Method VI: EDMD

In [ ]:
save_error = False

n_choices = 10                # Number of subsamples to take
n_steps   = len(t_vec_scen)            # No. timesteps
w0        = np.zeros(n_boxes) # Initial condition

# Basis functions
w_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
F_dict = utils_emulator.Vector_Dict(method='hermite', degree=1)
NRMSE_all_EDMD = utils_emulator.evaluate_ensemble('EDMD', z_ensemble, z_mean, None, rho, scenarios, n_ensemble,
                                                 n_choices, dt=dt, cubLor=True, rho_base=rho_base, t_vec=t_vec_scen,
                                                 baseline_mean=baseline_mean, n_boxes=n_boxes, w0=w0, n_steps=n_steps,
                                                 w_dict=w_dict, F_dict=F_dict)

if save_error:
  utils_emulator.save_results(NRMSE_all_EDMD, 'exp4_VI_EDMD_error_ensemble')